# IMPORT THE DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Adjust the path based on where you've stored the files
training_data = '/content/drive/MyDrive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/training_data.csv'
unlabelled_data = '/content/drive/MyDrive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/unlabelled_test_data.csv'
sample_submission = '/content/drive/MyDrive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/sample_submission.csv'

training_data_pd = pd.read_csv(training_data)
unlabelled_data_pd = pd.read_csv(unlabelled_data)
sample_submission_pd = pd.read_csv(sample_submission)


In [ ]:
display(training_data_pd)
display(unlabelled_data_pd)
display(sample_submission_pd)

,id,sentence,difficulty
0,0,les coûts kilométriques réels peuvent diverger...,C1
1,1,le bleu cest ma couleur préférée mais je naime...,A1
2,2,le test de niveau en français est sur le site ...,A1
3,3,estce que ton mari est aussi de boston,A1
4,4,dans les écoles de commerce dans les couloirs ...,B1
...,...,...,...
4795,4795,cest pourquoi il décida de remplacer les habit...,B2
4796,4796,il avait une de ces pâleurs splendides qui don...,C1
4797,4797,et le premier samedi de chaque mois venez renc...,A2
4798,4798,les coûts liés à la journalisation nétant pas ...,C2


,id,sentence
0,0,nous dûmes nous excuser des propos que nous eû...
1,1,vous ne pouvez pas savoir le plaisir que jai d...
2,2,et paradoxalement boire froid nest pas la bonn...
3,3,ce nest pas étonnant car cest une saison mysté...
4,4,le corps de golo luimême dune essence aussi su...
...,...,...
1195,1195,cest un phénomène qui trouve une accélération ...
1196,1196,je vais parler au serveur et voir si on peut d...
1197,1197,il nétait pas comme tant de gens qui par pares...
1198,1198,ils deviennent dangereux pour notre économie


,id,difficulty
0,0,A1
1,1,A1
2,2,A1
3,3,A1
4,4,A1
...,...,...
1195,1195,A1
1196,1196,A1
1197,1197,A1
1198,1198,A1


# BERT

In [ ]:
import pandas as pd
import string

# Function to clean text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra whitespace
    text = " ".join(text.split())
    return text

# Apply cleaning function to the sentence column in both datasets
training_data_pd['sentence'] = training_data_pd['sentence'].apply(clean_text)
unlabelled_data_pd['sentence'] = unlabelled_data_pd['sentence'].apply(clean_text)

# Display the first few rows of the cleaned data to confirm changes
print(training_data_pd.head())
print(unlabelled_data_pd.head())


   id                                           sentence difficulty
0   0  les coûts kilométriques réels peuvent diverger...         C1
1   1  le bleu cest ma couleur préférée mais je naime...         A1
2   2  le test de niveau en français est sur le site ...         A1
3   3             estce que ton mari est aussi de boston         A1
4   4  dans les écoles de commerce dans les couloirs ...         B1
   id                                           sentence
0   0  nous dûmes nous excuser des propos que nous eû...
1   1  vous ne pouvez pas savoir le plaisir que jai d...
2   2  et paradoxalement boire froid nest pas la bonn...
3   3  ce nest pas étonnant car cest une saison mysté...
4   4  le corps de golo luimême dune essence aussi su...


In [ ]:
"""
!pip install transformers
!pip install torch
"""
!pip install --no-cache-dir accelerate==0.29.3
!pip install --no-cache-dir transformers[torch]==4.40.1

import accelerate
print(accelerate.__version__)


0.29.3


In [ ]:
import accelerate
import transformers
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)


Accelerate version: 0.29.3
Transformers version: 4.40.1


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)  # Assuming 6 labels for CEFR levels A1, A2, B1, B2, C1, C2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pr

In [ ]:
# 52.5%

"""
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import numpy as np

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Function to encode data
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

# Metric for evaluation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=6,  # Slight adjustment if needed based on results
    per_device_train_batch_size=12,  # A bit smaller to adjust gradient updates
    warmup_steps=500,  # Adjust based on the total number of steps (num_epochs * total_data / batch_size)
    weight_decay=0.02,  # Good for regularization
    logging_dir='./logs',
    logging_steps=50,  # Less frequent to reduce logging overhead
    learning_rate=3e-5,  # Adjusted for potentially better convergence
    fp16=False,  # Make sure your hardware supports FP16 for this to be effective
    evaluation_strategy="epoch",  # Change to steps if you need more frequent evaluation
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# Initialize the Trainer with added compute_metrics for dynamic metric calculation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Ensure metrics are computed during evaluation
)

# Train the model
trainer.train()
"""

'\nfrom transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments\nfrom sklearn.model_selection import train_test_split\nfrom datasets import Dataset, load_metric\nimport numpy as np\n\n# Load the tokenizer\ntokenizer = DistilBertTokenizer.from_pretrained(\'distilbert-base-uncased\')\n\n# Function to encode data\ndef encode_data(tokenizer, df):\n    texts = df[\'sentence\'].tolist()\n    labels = df[\'difficulty\'].map({\'A1\': 0, \'A2\': 1, \'B1\': 2, \'B2\': 3, \'C1\': 4, \'C2\': 5}).tolist()\n    encodings = tokenizer(texts, truncation=True, padding=\'max_length\', max_length=128)\n    return Dataset.from_dict({\n        \'input_ids\': encodings[\'input_ids\'],\n        \'attention_mask\': encodings[\'attention_mask\'],\n        \'labels\': labels\n    })\n\n# Split the DataFrame into training and validation sets\ntrain_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)\n\n# Tokenize and prepare dataset

In [ ]:
pip install optuna


In [ ]:
pip install transformers datasets


In [ ]:
pip install transformers


In [ ]:
#Light CamemBERT first try, 56%
"""
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import numpy as np

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Function to encode data
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

# Load the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Metric for evaluation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # Slight adjustment if needed based on results
    per_device_train_batch_size=12,  # A bit smaller to adjust gradient updates
    warmup_steps=500,  # Adjust based on the total number of steps (num_epochs * total_data / batch_size)
    weight_decay=0.02,  # Good for regularization
    logging_dir='./logs',
    logging_steps=50,  # Less frequent to reduce logging overhead
    learning_rate=3e-5,  # Adjusted for potentially better convergence
    fp16=True,  # Make sure your hardware supports FP16 for this to be effective
    evaluation_strategy="epoch",  # Change to steps if you need more frequent evaluation
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# Initialize the Trainer with added compute_metrics for dynamic metric calculation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Ensure metrics are computed during evaluation
)

# Train the model
trainer.train()
"""
"""
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import numpy as np

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Function to encode data
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=512)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

# Load the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Metric for evaluation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,  # Slight adjustment if needed based on results
    per_device_train_batch_size=12,  # A bit smaller to adjust gradient updates
    warmup_steps=500,  # Adjust based on the total number of steps (num_epochs * total_data / batch_size)
    weight_decay=0.02,  # Good for regularization
    logging_dir='./logs',
    logging_steps=50,  # Less frequent to reduce logging overhead
    learning_rate=3e-5,  # Adjusted for potentially better convergence
    fp16=True,  # Make sure your hardware supports FP16 for this to be effective
    evaluation_strategy="epoch",  # Change to steps if you need more frequent evaluation
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# Initialize the Trainer with added compute_metrics for dynamic metric calculation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Ensure metrics are computed during evaluation
)

# Train the model
trainer.train()
"""


'\nfrom transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments\nfrom sklearn.model_selection import train_test_split\nfrom datasets import Dataset, load_metric\nimport numpy as np\n\n# Load the tokenizer\ntokenizer = CamembertTokenizer.from_pretrained(\'camembert-base\')\n\n# Function to encode data\ndef encode_data(tokenizer, df):\n    texts = df[\'sentence\'].tolist()\n    labels = df[\'difficulty\'].map({\'A1\': 0, \'A2\': 1, \'B1\': 2, \'B2\': 3, \'C1\': 4, \'C2\': 5}).tolist()\n    encodings = tokenizer(texts, truncation=True, padding=\'max_length\', max_length=512)\n    return Dataset.from_dict({\n        \'input_ids\': encodings[\'input_ids\'],\n        \'attention_mask\': encodings[\'attention_mask\'],\n        \'labels\': labels\n    })\n\n# Split the DataFrame into training and validation sets\ntrain_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)\n\n# Tokenize and prepare datasets\ntrain_dat

In [ ]:
import pandas as pd
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import numpy as np
"""
# Load your datasets
training_data_pd = pd.read_csv(training_data)
unlabelled_data_pd = pd.read_csv(unlabelled_data)
sample_submission_pd = pd.read_csv(sample_submission)

# Load the tokenizer and the model
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Encode data function
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)

    df_encoded = pd.DataFrame({
        'input_ids': list(encodings['input_ids']),
        'attention_mask': list(encodings['attention_mask']),
        'labels': labels
    })

    return Dataset.from_pandas(df_encoded)

# Compute metrics function
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {'accuracy': metric.compute(predictions=predictions, references=labels)['accuracy']}

# Grid search function
def grid_search(tokenizer, model, dataset, param_grid):
    results = []
    for batch_size in param_grid['batch_sizes']:
        for learning_rate in param_grid['learning_rates']:
            for test_size in param_grid['test_sizes']:
                train_data, val_data = train_test_split(dataset, test_size=test_size, random_state=42)
                train_dataset = encode_data(tokenizer, train_data)
                val_dataset = encode_data(tokenizer, val_data)

                training_args = TrainingArguments(
                    output_dir='./results',
                    num_train_epochs=7,
                    per_device_train_batch_size=batch_size,
                    learning_rate=learning_rate,
                    warmup_steps=500,
                    weight_decay=0.02,
                    logging_dir='./logs',
                    evaluation_strategy='epoch',
                    save_strategy='epoch',
                    load_best_model_at_end=True,
                    metric_for_best_model='accuracy',
                    greater_is_better=True,
                    fp16=True
                )

                trainer = Trainer(
                    model=model,
                    args=training_args,
                    train_dataset=train_dataset,
                    eval_dataset=val_dataset,
                    compute_metrics=compute_metrics,
                    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
                )

                trainer.train()

                eval_results = trainer.evaluate()
                results.append({
                    'batch_size': batch_size,
                    'learning_rate': learning_rate,
                    'test_size': test_size,
                    'accuracy': eval_results['eval_accuracy']
                })

    best_result = max(results, key=lambda x: x['accuracy'])
    return best_result, results

# Define parameter grid
param_grid = {
    'batch_sizes': [8, 16, 32],
    'learning_rates': [2e-5, 3e-5, 5e-5],
    'test_sizes': [0.1, 0.2, 0.3]
}

# Execute the grid search
best_parameters, all_results = grid_search(tokenizer, model, training_data_pd, param_grid)
print("Best Parameters:", best_parameters)
"""

'\n# Load your datasets\ntraining_data_pd = pd.read_csv(training_data)\nunlabelled_data_pd = pd.read_csv(unlabelled_data)\nsample_submission_pd = pd.read_csv(sample_submission)\n\n# Load the tokenizer and the model\ntokenizer = CamembertTokenizer.from_pretrained(\'camembert-base\')\nmodel = CamembertForSequenceClassification.from_pretrained(\'camembert-base\', num_labels=6)\n\n# Encode data function\ndef encode_data(tokenizer, df):\n    texts = df[\'sentence\'].tolist()\n    labels = df[\'difficulty\'].map({\'A1\': 0, \'A2\': 1, \'B1\': 2, \'B2\': 3, \'C1\': 4, \'C2\': 5}).tolist()\n    encodings = tokenizer(texts, truncation=True, padding=\'max_length\', max_length=128)\n\n    df_encoded = pd.DataFrame({\n        \'input_ids\': list(encodings[\'input_ids\']),\n        \'attention_mask\': list(encodings[\'attention_mask\']),\n        \'labels\': labels\n    })\n\n    return Dataset.from_pandas(df_encoded)\n\n# Compute metrics function\ndef compute_metrics(eval_pred):\n    metric = load

In [ ]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import pandas as pd
import numpy as np
"""
# Load the tokenizer and the model
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Encode data function
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)

    df_encoded = pd.DataFrame({
        'input_ids': list(encodings['input_ids']),
        'attention_mask': list(encodings['attention_mask']),
        'labels': labels
    })

    return Dataset.from_pandas(df_encoded)

# Compute metrics function
def compute_metrics(eval_pred):
    metric = load_metric("accuracy", trust_remote_code=True)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {'accuracy': metric.compute(predictions=predictions, references=labels)['accuracy']}

# Train and validation split
train_data, val_data = train_test_split(training_data_pd, test_size=0.1, random_state=42)
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

# Training arguments using the best parameters
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=6,
    per_device_train_batch_size=16,
    learning_rate=3e-5,
    warmup_steps=500,
    weight_decay=0.02,
    logging_dir='./logs',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    fp16=True
)

# Initialize Trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train the model
trainer.train()

# Evaluation on the validation set
eval_results = trainer.evaluate()
print("Validation Results:", eval_results)
"""

'\n# Load the tokenizer and the model\ntokenizer = CamembertTokenizer.from_pretrained(\'camembert-base\')\nmodel = CamembertForSequenceClassification.from_pretrained(\'camembert-base\', num_labels=6)\n\n# Encode data function\ndef encode_data(tokenizer, df):\n    texts = df[\'sentence\'].tolist()\n    labels = df[\'difficulty\'].map({\'A1\': 0, \'A2\': 1, \'B1\': 2, \'B2\': 3, \'C1\': 4, \'C2\': 5}).tolist()\n    encodings = tokenizer(texts, truncation=True, padding=\'max_length\', max_length=128)\n\n    df_encoded = pd.DataFrame({\n        \'input_ids\': list(encodings[\'input_ids\']),\n        \'attention_mask\': list(encodings[\'attention_mask\']),\n        \'labels\': labels\n    })\n\n    return Dataset.from_pandas(df_encoded)\n\n# Compute metrics function\ndef compute_metrics(eval_pred):\n    metric = load_metric("accuracy", trust_remote_code=True)\n    logits, labels = eval_pred\n    predictions = np.argmax(logits, axis=1)\n    return {\'accuracy\': metric.compute(predictions=pr

In [ ]:
#CamemBERT 57.5%
"""
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import numpy as np

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Function to encode data
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

# Load the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Metric for evaluation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=9,  # Slight adjustment if needed based on results
    per_device_train_batch_size=12,  # A bit smaller to adjust gradient updates
    warmup_steps=500,  # Adjust based on the total number of steps (num_epochs * total_data / batch_size)
    weight_decay=0.02,  # Good for regularization
    logging_dir='./logs',
    logging_steps=50,  # Less frequent to reduce logging overhead
    learning_rate=3e-5,  # Adjusted for potentially better convergence
    fp16=True,  # Make sure your hardware supports FP16 for this to be effective
    evaluation_strategy="epoch",  # Change to steps if you need more frequent evaluation
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# Initialize the Trainer with added compute_metrics for dynamic metric calculation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Ensure metrics are computed during evaluation
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]

)

# Train the model
trainer.train()
"""
#59%
"""
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import numpy as np

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Function to encode data
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

# Load the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Metric for evaluation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=6,  # Slight adjustment if needed based on results
    per_device_train_batch_size=22,  # A bit smaller to adjust gradient updates
    warmup_steps=500,  # Adjust based on the total number of steps (num_epochs * total_data / batch_size)
    weight_decay=0.01,  # Good for regularization
    logging_dir='./logs',
    logging_steps=50,  # Less frequent to reduce logging overhead
    learning_rate=15e-5,  # Adjusted for potentially better convergence
    fp16=True,  # Make sure your hardware supports FP16 for this to be effective
    evaluation_strategy="epoch",  # Change to steps if you need more frequent evaluation
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# Initialize the Trainer with added compute_metrics for dynamic metric calculation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Ensure metrics are computed during evaluation
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]

)

# Train the model
trainer.train()
"""


from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import numpy as np

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Function to encode data
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

# Load the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Metric for evaluation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,  # Slight adjustment if needed based on results
    per_device_train_batch_size=32,  # A bit smaller to adjust gradient updates
    warmup_steps=500,  # Adjust based on the total number of steps (num_epochs * total_data / batch_size)
    weight_decay=0.01,  # Good for regularization
    logging_dir='./logs',
    logging_steps=50,  # Less frequent to reduce logging overhead
    learning_rate=18e-5,  # Adjusted for potentially better convergence
    fp16=True,  # Make sure your hardware supports FP16 for this to be effective
    evaluation_strategy="epoch",  # Change to steps if you need more frequent evaluation
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# Initialize the Trainer with added compute_metrics for dynamic metric calculation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Ensure metrics are computed during evaluation
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]

)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task t

Epoch,Training Loss,Validation Loss,Accuracy
1,1.612400,1.278122,0.485417


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:

from transformers import CamembertTokenizer
import numpy as np

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Tokenize the sentences from the 'sentence' column, specifying max_length
max_length = 512  # Typical max length for BERT-based models
unlabelled_encodings = tokenizer(list(unlabelled_data_pd['sentence']), truncation=True, padding=True, max_length=max_length, return_tensors="pt")

# Assuming 'trainer' is your trained model's Trainer instance
from torch.utils.data import Dataset, DataLoader

class SimpleDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# Create a dataset from the tokenized data
dataset = SimpleDataset(unlabelled_encodings)

# Predict using the trainer
predictions = trainer.predict(dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Define a mapping from numeric labels to categories
difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in predicted_labels]

import pandas as pd

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predicted_difficulties
})

# Save the DataFrame to a CSV file for submission
submission_df.to_csv('/content/drive/My Drive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/submission_CamemBERT_12.csv', index=False)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

!ls -lh

!rm -rf /content/logs

!rm -rf /content/results

!rm -rf /content/sample_data

!ls -lh


total 52K
drwxr-xr-x  2 root root 4.0K May  8 12:48 camembert-finetuned-batch_size_16
drwxr-xr-x  2 root root 4.0K May  8 12:51 camembert-finetuned-batch_size_18
drwxr-xr-x  2 root root 4.0K May  8 12:55 camembert-finetuned-batch_size_20
drwxr-xr-x  2 root root 4.0K May  8 13:00 camembert-finetuned-batch_size_22
drwxr-xr-x  2 root root 4.0K May  8 13:03 camembert-finetuned-batch_size_24
drwxr-xr-x  2 root root 4.0K May  8 13:43 camembert-finetuned-batch_size_26
drwxr-xr-x  2 root root 4.0K May  8 13:46 camembert-finetuned-batch_size_32
drwxr-xr-x  2 root root 4.0K May  8 13:50 camembert-finetuned-batch_size_38
drwxr-xr-x  2 root root 4.0K May  8 13:53 camembert-finetuned-batch_size_48
drwx------  6 root root 4.0K May  8 12:25 drive
drwxr-xr-x  2 root root 4.0K May  8 15:32 logs
drwxr-xr-x 58 root root 4.0K May  8 15:37 results
drwxr-xr-x  1 root root 4.0K May  6 13:19 sample_data
total 40K
drwxr-xr-x 2 root root 4.0K May  8 12:48 camembert-finetuned-batch_size_16
drwxr-xr-x 2 root root